In [6]:
import sys
from pathlib import Path
import warnings
warnings.filterwarnings("ignore", module="IPython")

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

print(f"Root dir: {root_dir}")

# Add the root directory to the `PYTHONPATH` 
if root_dir not in sys.path:
    sys.path.append(root_dir)
    print(f"Added the following directory to the PYTHONPATH: {root_dir}")

# Set the environment variables from the file <root_dir>/.env
from mlfs import config
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Root dir: c:\Users\nives\AllFolders\SML\mlfs-book
HopsworksSettings initialized!


### <span style='color:#ff5f27'>Imports

In [7]:
import datetime
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
import datetime
from pathlib import Path
import json
import re
import os
import warnings
warnings.filterwarnings("ignore")

### Login into Hopsworks


In [8]:
project = hopsworks.login()

2025-11-14 22:28:44,826 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-11-14 22:28:44,830 INFO: Initializing external client
2025-11-14 22:28:44,830 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-14 22:28:46,018 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279175


## <span style='color:#ff5f27'> Get the AQICN_URL and API key. Enter country, city, street names for your Sensor.</span>

In [9]:
today = datetime.date.today()

csv_file=f"{root_dir}/data/air-quality-data.csv"
util.check_file_path(csv_file)

# taken from ~/.env.
if settings.AQICN_API_KEY is None:
    print("You need to set AQICN_API_KEY either in this cell or in ~/.env")
    sys.exit(1)

AQICN_API_KEY = settings.AQICN_API_KEY.get_secret_value() 
aqicn_url = "https://api.waqi.info/feed/@13983"
country = "Sweden"
city = "Sollentuna"
street = "ekmans-vag-11"
########################latitude, longitude = util.get_city_coordinates(city)

print(f"Found AQICN_API_KEY: {AQICN_API_KEY}")

secrets = hopsworks.get_secrets_api()

# Replace any existing secret with the new value
secret = secrets.get_secret("AQICN_API_KEY")
if secret is not None:
    secret.delete()
    print("Replacing existing AQICN_API_KEY")

secrets.create_secret("AQICN_API_KEY", AQICN_API_KEY)

File successfully found at the path: c:\Users\nives\AllFolders\SML\mlfs-book/data/air-quality-data.csv
Found AQICN_API_KEY: b7b6ef9c10e47611f567a3080c006412df782712
Replacing existing AQICN_API_KEY
Secret created successfully, explore it at https://c.app.hopsworks.ai:443/account/secrets


Secret('AQICN_API_KEY', 'PRIVATE')

In [10]:
# Uncomment if 
# latitude, longitude = util.get_city_coordinates(city)ù
# results in an error

# Coordinates 59.43760681152344°N 18.04100227355957°E
# Elevation 26.0 m asl

latitude = 59.43
longitude = 17.95

### Validate that the AQICN_API_KEY works

In [11]:
try:
    aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
except hopsworks.RestAPIError:
    print("It looks like the AQICN_API_KEY doesn't work for your sensor. Is the API key correct? Is the sensor URL correct?")

aq_today_df.head()

,pm25,country,city,street,date,url
0,10.0,Sweden,Sollentuna,ekmans-vag-11,2025-11-14,https://api.waqi.info/feed/@13983


## <span style='color:#ff5f27'> Read your CSV file into a DataFrame </span>

In [12]:
df = pd.read_csv(csv_file,  parse_dates=['date'], skipinitialspace=True)
df

,date,pm25,pm10
0,2025-11-01,8.0,10.0
1,2025-11-02,27.0,7.0
2,2025-11-03,17.0,4.0
3,2025-11-04,10.0,7.0
4,2025-11-05,16.0,9.0
...,...,...,...
1140,2024-07-03,NaN,7.0
1141,2024-01-22,NaN,4.0
1142,2023-10-10,NaN,5.0
1143,2023-04-04,NaN,4.0


## <span style='color:#ff5f27'> Data cleaning</span>

We want to have a DataFrame with 2 columns - `date` and `pm25` after this cell below:

### Check the data types for the columns in your DataFrame

 * `date` should be of type   datetime64[ns] 
 * `pm25` should be of type float64

In [13]:
df_aq = df[['date', 'pm25']]
df_aq['pm25'] = df_aq['pm25'].astype('float32')

df_aq

,date,pm25
0,2025-11-01,8.0
1,2025-11-02,27.0
2,2025-11-03,17.0
3,2025-11-04,10.0
4,2025-11-05,16.0
...,...,...
1140,2024-07-03,NaN
1141,2024-01-22,NaN
1142,2023-10-10,NaN
1143,2023-04-04,NaN


### Adding lagged features

In [14]:
lagged_days = 3

df_aq_lagged = pd.DataFrame()
for i in range(1,lagged_days+1):
    df_aq_lagged['date'] = df_aq['date'] + datetime.timedelta(days=i)
    df_aq_lagged['pm25'] = df_aq['pm25']
    suffix='_lagged_' + str(i)
    df_aq = df_aq.merge(df_aq_lagged, on='date', how='left', suffixes=['',suffix])

df_aq

,date,pm25,pm25_lagged_1,pm25_lagged_2,pm25_lagged_3
0,2025-11-01,8.0,7.0,15.0,14.0
1,2025-11-02,27.0,8.0,7.0,15.0
2,2025-11-03,17.0,27.0,8.0,7.0
3,2025-11-04,10.0,17.0,27.0,8.0
4,2025-11-05,16.0,10.0,17.0,27.0
...,...,...,...,...,...
1140,2024-07-03,NaN,NaN,NaN,NaN
1141,2024-01-22,NaN,NaN,NaN,NaN
1142,2023-10-10,NaN,6.0,4.0,4.0
1143,2023-04-04,NaN,NaN,NaN,NaN


In [15]:
# Cast the pm25 column to be a float32 data type
df_aq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1145 entries, 0 to 1144
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           1145 non-null   datetime64[ns]
 1   pm25           1137 non-null   float32       
 2   pm25_lagged_1  1131 non-null   float32       
 3   pm25_lagged_2  1126 non-null   float32       
 4   pm25_lagged_3  1121 non-null   float32       
dtypes: datetime64[ns](1), float32(4)
memory usage: 27.0 KB


### Drop any rows with missing data </span>
It will make the model training easier if there is no missing data in the rows, so we drop any rows with missing data.

In [16]:
df_aq.dropna(inplace=True)
df_aq

,date,pm25,pm25_lagged_1,pm25_lagged_2,pm25_lagged_3
0,2025-11-01,8.0,7.0,15.0,14.0
1,2025-11-02,27.0,8.0,7.0,15.0
2,2025-11-03,17.0,27.0,8.0,7.0
3,2025-11-04,10.0,17.0,27.0,8.0
4,2025-11-05,16.0,10.0,17.0,27.0
...,...,...,...,...,...
1132,2022-09-26,18.0,21.0,20.0,15.0
1133,2022-09-27,28.0,18.0,21.0,20.0
1134,2022-09-28,19.0,28.0,18.0,21.0
1135,2022-09-29,12.0,19.0,28.0,18.0


### Add country, city, street, url to the DataFrame

We add the columns for the country, city, and street names that you changed for your Air Quality sensor.

We also want to make sure the `pm25` column is a float32 data type.

In [17]:
# Keep the date and pm25 columns
# If the column names in your DataFrame are different, rename your columns to `date` and `pm25`
df_aq['country']=country
df_aq['city']=city
df_aq['street']=street
df_aq['url']=aqicn_url
df_aq

,date,pm25,pm25_lagged_1,pm25_lagged_2,pm25_lagged_3,country,city,street,url
0,2025-11-01,8.0,7.0,15.0,14.0,Sweden,Sollentuna,ekmans-vag-11,https://api.waqi.info/feed/@13983
1,2025-11-02,27.0,8.0,7.0,15.0,Sweden,Sollentuna,ekmans-vag-11,https://api.waqi.info/feed/@13983
2,2025-11-03,17.0,27.0,8.0,7.0,Sweden,Sollentuna,ekmans-vag-11,https://api.waqi.info/feed/@13983
3,2025-11-04,10.0,17.0,27.0,8.0,Sweden,Sollentuna,ekmans-vag-11,https://api.waqi.info/feed/@13983
4,2025-11-05,16.0,10.0,17.0,27.0,Sweden,Sollentuna,ekmans-vag-11,https://api.waqi.info/feed/@13983
...,...,...,...,...,...,...,...,...,...
1132,2022-09-26,18.0,21.0,20.0,15.0,Sweden,Sollentuna,ekmans-vag-11,https://api.waqi.info/feed/@13983
1133,2022-09-27,28.0,18.0,21.0,20.0,Sweden,Sollentuna,ekmans-vag-11,https://api.waqi.info/feed/@13983
1134,2022-09-28,19.0,28.0,18.0,21.0,Sweden,Sollentuna,ekmans-vag-11,https://api.waqi.info/feed/@13983
1135,2022-09-29,12.0,19.0,28.0,18.0,Sweden,Sollentuna,ekmans-vag-11,https://api.waqi.info/feed/@13983


In [18]:
df_aq.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1113 entries, 0 to 1136
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           1113 non-null   datetime64[ns]
 1   pm25           1113 non-null   float32       
 2   pm25_lagged_1  1113 non-null   float32       
 3   pm25_lagged_2  1113 non-null   float32       
 4   pm25_lagged_3  1113 non-null   float32       
 5   country        1113 non-null   object        
 6   city           1113 non-null   object        
 7   street         1113 non-null   object        
 8   url            1113 non-null   object        
dtypes: datetime64[ns](1), float32(4), object(4)
memory usage: 69.6+ KB


## <span style='color:#ff5f27'> Loading Weather Data from [Open Meteo](https://open-meteo.com/en/docs)

### Download the Historical Weather Data

https://open-meteo.com/en/docs/historical-weather-api#hourly=&daily=temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant

We will download the historical weather data for your `city` by first extracting the earliest date from your DataFrame containing the historical air quality measurements.

We will download all daily historical weather data measurements for your `city` from the earliest date in your air quality measurement DataFrame. It doesn't matter if there are missing days of air quality measurements. We can store all of the daily weather measurements, and when we build our training dataset, we will join up the air quality measurements for a given day to its weather features for that day. 

The weather features we will download are:

 * `temperature (average over the day)`
 * `precipitation (the total over the day)`
 * `wind speed (average over the day)`
 * `wind direction (the most dominant direction over the day)`


In [19]:
earliest_aq_date = pd.Series.min(df_aq['date'])
earliest_aq_date = earliest_aq_date.strftime('%Y-%m-%d')
earliest_aq_date

weather_df = util.get_historical_weather(city, earliest_aq_date, str(today), latitude, longitude)

Coordinates 59.43760681152344°N 18.04100227355957°E
Elevation 26.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [20]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         1167 non-null   datetime64[ns]
 1   temperature_2m_mean          1167 non-null   float32       
 2   precipitation_sum            1167 non-null   float32       
 3   wind_speed_10m_max           1167 non-null   float32       
 4   wind_direction_10m_dominant  1167 non-null   float32       
 5   city                         1167 non-null   object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 36.6+ KB


## <span style='color:#ff5f27'> Define Data Validation Rules </span>

### Expectations for Weather Data
We will validate the air quality measurements (`pm25` values) before we write them to Hopsworks.

We define a data validation rule (an expectation in Great Expectations) that ensures that `pm25` values are not negative or above the max value available by the sensor.

We will attach this expectation to the air quality feature group, so that we validate the `pm25` data every time we write a DataFrame to the feature group. We want to prevent garbage-in, garbage-out.


In [21]:
import great_expectations as ge
aq_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="aq_expectation_suite"
)

aq_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column":"pm25",
            "min_value":-0.1,
            "max_value":500.0,
            "strict_min":True
        }
    )
)

{"expectation_type": "expect_column_min_to_be_between", "kwargs": {"column": "pm25", "min_value": -0.1, "max_value": 500.0, "strict_min": true}, "meta": {}}

### Expectations for Weather Data
Here, we define an expectation for 2 columns in our weather DataFrame - `precipitation_sum` and `wind_speed_10m_max`, where we expect both values to be greater than zero, but less than 1000.

In [22]:
import great_expectations as ge
weather_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="weather_expectation_suite"
)

def expect_greater_than_zero(col):
    weather_expectation_suite.add_expectation(
        ge.core.ExpectationConfiguration(
            expectation_type="expect_column_min_to_be_between",
            kwargs={
                "column":col,
                "min_value":-0.1,
                "max_value":1000.0,
                "strict_min":True
            }
        )
    )
expect_greater_than_zero("precipitation_sum")
expect_greater_than_zero("wind_speed_10m_max")

---

### <span style="color:#ff5f27;"> Connect to Hopsworks and save the sensor country, city, street names as a secret</span>

In [23]:
fs = project.get_feature_store() 

#### Save country, city, street names as a secret

These will be downloaded from Hopsworks later in the (1) daily feature pipeline and (2) the daily batch inference pipeline

In [24]:
dict_obj = {
    "country": country,
    "city": city,
    "street": street,
    "aqicn_url": aqicn_url,
    "latitude": latitude,
    "longitude": longitude
}

# Convert the dictionary to a JSON string
str_dict = json.dumps(dict_obj)

# Replace any existing secret with the new value
secret = secrets.get_secret("SENSOR_LOCATION_JSON")
if secret is not None:
    secret.delete()
    print("Replacing existing SENSOR_LOCATION_JSON")

secrets.create_secret("SENSOR_LOCATION_JSON", str_dict)

Replacing existing SENSOR_LOCATION_JSON
Secret created successfully, explore it at https://c.app.hopsworks.ai:443/account/secrets


Secret('SENSOR_LOCATION_JSON', 'PRIVATE')

### <span style="color:#ff5f27;"> Create the Feature Groups and insert the DataFrames in them </span>

### Air Quality Data
    
 1. Provide a name, description, and version for the feature group.
 2. Define the `primary_key`: we have to select which columns uniquely identify each row in the DataFrame - by providing them as the `primary_key`. Here, each air quality sensor measurement is uniquely identified by `country`, `street`, and  `date`.
 3. Define the `event_time`: We also define which column stores the timestamp or date for the row - `date`.
 4. Attach any `expectation_suite` containing data validation rules

In [25]:
air_quality_fg = fs.get_or_create_feature_group(
    name='air_quality',
    description='Air Quality characteristics of each day',
    version=3,
    primary_key=['country','city', 'street'],
    event_time="date",
    expectation_suite=aq_expectation_suite
)

#### Insert the DataFrame into the Feature Group

In [26]:
air_quality_fg.insert(df_aq)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1279175/fs/1265791/fg/1717615
2025-11-14 22:30:32,905 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279175/fs/1265791/fg/1717615


Uploading Dataframe: 100.00% |██████████| Rows 1113/1113 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: air_quality_3_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279175/jobs/named/air_quality_3_offline_fg_materialization/executions


(Job('air_quality_3_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 757791
         }
       },
       "result": {
         "observed_value": 4.0,
         "element_count": 1113,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-14T09:30:32.000905Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expect

#### Enter a description for each feature in the Feature Group

In [28]:
air_quality_fg.update_feature_description("date", "Date of measurement of air quality")
air_quality_fg.update_feature_description("country", "Country where the air quality was measured (sometimes a city in acqcn.org)")
air_quality_fg.update_feature_description("city", "City where the air quality was measured")
air_quality_fg.update_feature_description("street", "Street in the city where the air quality was measured")
air_quality_fg.update_feature_description("pm25", "Particles less than 2.5 micrometers in diameter (fine particles) pose health risk")

air_quality_fg.update_feature_description("pm25_lagged_1", "pm25 value measured 1 day before the given date")
air_quality_fg.update_feature_description("pm25_lagged_2", "pm25 value measured 2 days before the given date")
air_quality_fg.update_feature_description("pm25_lagged_3", "pm25 value measured 3 days before the given date")

### Weather Data
    
 1. Provide a name, description, and version for the feature group.
 2. Define the `primary_key`: we have to select which columns uniquely identify each row in the DataFrame - by providing them as the `primary_key`. Here, each weather measurement is uniquely identified by `city` and  `date`.
 3. Define the `event_time`: We also define which column stores the timestamp or date for the row - `date`.
 4. Attach any `expectation_suite` containing data validation rules

In [29]:
# Get or create feature group 
weather_fg = fs.get_or_create_feature_group(
    name='weather',
    description='Weather characteristics of each day',
    version=1,
    primary_key=['city'],
    event_time="date",
    expectation_suite=weather_expectation_suite
) 

#### Insert the DataFrame into the Feature Group

In [30]:
# Insert data
weather_fg.insert(weather_df, wait=True)

2025-11-14 22:34:13,216 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279175/fs/1265791/fg/1703315


Uploading Dataframe: 100.00% |██████████| Rows 1167/1167 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279175/jobs/named/weather_1_offline_fg_materialization/executions
2025-11-14 22:34:30,295 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-14 22:34:33,498 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-14 22:36:22,639 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-14 22:36:22,800 INFO: Waiting for log aggregation to finish.
2025-11-14 22:36:31,445 INFO: Execution finished successfully.


(Job('weather_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 747580
         }
       },
       "result": {
         "observed_value": 4.916339874267578,
         "element_count": 1167,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-14T09:34:13.000214Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type

#### Enter a description for each feature in the Feature Group

In [31]:
weather_fg.update_feature_description("date", "Date of measurement of weather")
weather_fg.update_feature_description("city", "City where weather is measured/forecast for")
weather_fg.update_feature_description("temperature_2m_mean", "Temperature in Celsius")
weather_fg.update_feature_description("precipitation_sum", "Precipitation (rain/snow) in mm")
weather_fg.update_feature_description("wind_speed_10m_max", "Wind speed at 10m abouve ground")
weather_fg.update_feature_description("wind_direction_10m_dominant", "Dominant Wind direction over the dayd")